In [1]:
import pandas as pd
import numpy as np

In [2]:
df_ila = pd.read_csv('obfuscated_item_location_assignment.txt', sep = '\t')

In [3]:
# get current total sale and number of locations per station
df_sumsales = df_ila.groupby(['workstation'], as_index=False).agg({
    'sales': 'sum',
    'location': 'count'
})

df_sumsales['diffe'] = df_sumsales['sales'] - int(df_sumsales.sales.mean())

sum(np.abs(df_sumsales.diffe)), sum(df_sumsales.sales)

(1389748, 1557926)

In [4]:
# get number of location per station, for initial assignment
station_count_location_mapping = df_sumsales[['workstation','location']]
station_count_location_mapping.columns = ['workstation','num_location']
station_count_location_mapping.sort_values(by = 'num_location', inplace=True)
station_count_location_mapping['num_location_cumsum'] = np.cumsum(station_count_location_mapping.num_location)
station_count_location_mapping.reset_index(inplace=True, drop=True)

# mapping item, sales and corresponding 'index', for future use
dic_index_item_mapping = {}
dic_index_sales_mapping = {}
dic_item_sales_mapping = {}

for row in df_ila.iterrows():
    item_id = row[1][0]
    sale = row[1][1]
    index = row[0]
    dic_index_item_mapping[index] = item_id
    dic_index_sales_mapping[index] = sale
    dic_item_sales_mapping[item_id] = sale

# initial assignment
item_sales_sortlist = sorted(df_ila.sales, reverse=True)
item_index_sortlist = sorted(range(len(df_ila.sales)), key=lambda k: df_ila.sales[k], reverse = True)

tmp_sales = []
tmp_sales_sum = []
tmp_index = []
previous = 0
for i,cnt in enumerate(station_count_location_mapping.num_location_cumsum): 
    tmp_sales.append([item for item in item_sales_sortlist[previous:cnt]])
    tmp_sales_sum.append(sum([item for item in item_sales_sortlist[previous:cnt]]))
    tmp_index.append([item for item in item_index_sortlist[previous:cnt]])
    previous = cnt

sales_initial_mapping = pd.DataFrame()
sales_initial_mapping['initial_sum'] = tmp_sales_sum
sales_initial_mapping['initial_indi_sum'] = tmp_sales
sales_initial_mapping['initial_indi_index'] = tmp_index

# first time adjustment
tmp_sales_sum_sorted = sorted(tmp_sales_sum, reverse=True)

dic_initial_sales_sum = {}
for k,v in enumerate(tmp_sales_sum):
    dic_initial_sales_sum[v] = k  # create dictionary for future use
    
dic_initial_sales = {}
for k,v in enumerate(tmp_sales):
    dic_initial_sales[k] = v
    
dic_initial_index = {}
for k,v in enumerate(tmp_index):
    dic_initial_index[k] = v
    
mean_sales = int(df_sumsales.sales.mean()) # 'best' average sales per station

# based on the assumption that after initial assignment, there are always less than half of stations with total sales
# more than 'best' average sales per station 
num_adjust = len(sales_initial_mapping[sales_initial_mapping.initial_sum > mean_sales])

new_plan_group1 = {}
new_plan_group2 = {}
new_plan_group1_sum = {}
new_plan_group2_sum = {}
new_plan_group1_index = {}
new_plan_group2_index = {}
index1_used = []
index2_used = []
index_used = []

cur_imbalance_list = []
for j in range(num_adjust):
    ind1 = dic_initial_sales_sum[tmp_sales_sum_sorted[j]]  # index of station considered adjustment in jth round
    ind2 = dic_initial_sales_sum[tmp_sales_sum_sorted[-j-1]]   # index of station considered adjustment in jth round
    print('group1:',j, ind1)
    print('group2:',-j-1, ind2)
    index1_used += [ind1]
    index2_used += [ind2]
    index_used += [ind1, ind2]
    
    cur_imbalance = mean_sales - tmp_sales_sum_sorted[-j-1]
    #print(cur_imbalance)
    i = 0
    new_plan_ind1 = []
    new_plan_ind2 = []
    new_plan_item_index1 = []
    new_plan_item_index2 = []
    max_adjust_length = min(len(dic_initial_sales[ind1]), len(dic_initial_sales[ind2]))
    
    if max_adjust_length > 1: 
        while cur_imbalance > 0:
            
            if i < max_adjust_length - 1:
                new_plan_ind2.append(dic_initial_sales[ind1][-i-1]) # switch item
                new_plan_ind1.append(dic_initial_sales[ind2][i])
                new_plan_item_index2.append(dic_initial_index[ind1][-i-1])
                new_plan_item_index1.append(dic_initial_index[ind2][i])
                cur_imbalance -= dic_initial_sales[ind1][-i-1] - dic_initial_sales[ind2][i]
                i += 1
            else: break   
            
    new_plan_ind2 = new_plan_ind2 + dic_initial_sales[ind2][i:]  
    new_plan_item_index2 = new_plan_item_index2 + dic_initial_index[ind2][i:]  # store index of new items in station
    if i == 0: 
        new_plan_ind1 = dic_initial_sales[ind1]
        new_plan_item_index1 = dic_initial_index[ind1]
    else: 
        new_plan_ind1 = new_plan_ind1 + dic_initial_sales[ind1][:-i]
        new_plan_item_index1 = new_plan_item_index1 + dic_initial_index[ind1][:-i]
    
    new_plan_group1[ind1] = sorted(new_plan_ind1, reverse=True)
    new_plan_group2[ind2] = sorted(new_plan_ind2, reverse=True)
    new_plan_group1_index[ind1] = sorted(new_plan_item_index1, reverse=True)
    new_plan_group2_index[ind2] = sorted(new_plan_item_index2, reverse=True)
    
    new_plan_group1_sum[ind1] = sum(new_plan_ind1)
    new_plan_group2_sum[ind2] = sum(new_plan_ind2)
    
    cur_imbalance_list.append(cur_imbalance)
    print(i, cur_imbalance)
    
# assignment after first adjustment    
sales_new_mapping = sales_initial_mapping.copy()
for ind in index1_used:
    sales_new_mapping['initial_sum'].iloc[ind] = new_plan_group1_sum[ind]
    sales_new_mapping['initial_indi_sum'].iloc[ind] = new_plan_group1[ind]
    sales_new_mapping['initial_indi_index'].iloc[ind] = new_plan_group1_index[ind]

for ind in index2_used:
    sales_new_mapping['initial_sum'].iloc[ind] = new_plan_group2_sum[ind]
    sales_new_mapping['initial_indi_sum'].iloc[ind] = new_plan_group2[ind]
    sales_new_mapping['initial_indi_index'].iloc[ind] = new_plan_group2_index[ind]

sales_new_mapping_sorted = sales_new_mapping.sort_values(by='initial_sum')

# approximate initial total imbalance
total_imbalance_last_round = 770000

/Users/chao_zhuo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/chao_zhuo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


group1: 0 19
group2: -1 59
12 -172
group1: 1 20
group2: -2 58
12 -101
group1: 2 21
group2: -3 54
13 -734
group1: 3 22
group2: -4 53
15 -875
group1: 4 23
group2: -5 50
16 -74
group1: 5 25
group2: -6 51
21 -536
group1: 6 24
group2: -7 52
18 -945
group1: 7 26
group2: -8 49
24 -239
group1: 8 27
group2: -9 47
28 -315
group1: 9 18
group2: -10 48
7 -892
group1: 10 2
group2: -11 57
2 -2406
group1: 11 28
group2: -12 46
31 -19
group1: 12 31
group2: -13 0
0 14552
group1: 13 29
group2: -14 45
35 -68
group1: 14 3
group2: -15 44
2 -1906
group1: 15 32
group2: -16 43
49 -35
group1: 16 30
group2: -17 42
35 -53
group1: 17 33
group2: -18 41
53 -128
group1: 18 5
group2: -19 56
2 -1074
group1: 19 4
group2: -20 40
2 -2976


/Users/chao_zhuo/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


###  run the cell below two times

In [6]:
### 2nd and 3rd adjustment

remove_start = 1 # ignore the station with only one location
sales_new_mapping_remove1 = sales_new_mapping[remove_start:]

tmp_index = sales_new_mapping_remove1['initial_indi_index']
tmp_sales = sales_new_mapping_remove1['initial_indi_sum']
tmp_sales_sum = sales_new_mapping_remove1['initial_sum']
tmp_sales_sum_sorted = sorted(tmp_sales_sum, reverse=True)

dic_initial_sales_sum = {}
for k,v in enumerate(tmp_sales_sum):
    dic_initial_sales_sum[v] = k+remove_start 
    
dic_initial_sales = {}
for k,v in enumerate(tmp_sales):
    dic_initial_sales[k+remove_start] = v

dic_initial_index = {}
for k,v in enumerate(tmp_index):
    dic_initial_index[k+remove_start] = v
    
num_adjust = min(len(sales_new_mapping_remove1[sales_new_mapping_remove1.initial_sum > mean_sales]), \
                 len(sales_new_mapping_remove1[sales_new_mapping_remove1.initial_sum < mean_sales]))

new_plan_group1 = {}
new_plan_group2 = {}
new_plan_group1_sum = {}
new_plan_group2_sum = {}
new_plan_group1_index = {}
new_plan_group2_index = {}
index1_used = []
index2_used = []
index_used = []

cur_imbalance_list = []
for j in range(num_adjust):
    ind1 = dic_initial_sales_sum[tmp_sales_sum_sorted[j]]
    ind2 = dic_initial_sales_sum[tmp_sales_sum_sorted[-j-1]]
    print('group1:',j, ind1)
    print('group2:',-j-1, ind2)
    index1_used += [ind1]
    index2_used += [ind2]
    index_used += [ind1, ind2]
    
    cur_imbalance = mean_sales - tmp_sales_sum_sorted[-j-1]
    print(cur_imbalance)
    i = 0
    new_plan_ind1 = []
    new_plan_ind2 = []
    new_plan_item_index1 = []
    new_plan_item_index2 = []
    max_adjust_length = min(len(dic_initial_sales[ind1]), len(dic_initial_sales[ind2]))
    
    if max_adjust_length > 1: 
        while cur_imbalance > 0:
            
            if i < max_adjust_length:
                
                if dic_initial_sales[ind1][i] > dic_initial_sales[ind2][-i-1]:
                    # difference compared to the first adjustment, in terms of item chosen
                    new_plan_ind2.append(dic_initial_sales[ind1][i])                                                    
                    new_plan_ind1.append(dic_initial_sales[ind2][-i-1])
                    new_plan_item_index2.append(dic_initial_index[ind1][i])
                    new_plan_item_index1.append(dic_initial_index[ind2][-i-1])
                    cur_imbalance -= dic_initial_sales[ind1][i] - dic_initial_sales[ind2][-i-1]
                    i += 1
                else: 
                    new_plan_ind1.append(dic_initial_sales[ind1][i])
                    new_plan_ind2.append(dic_initial_sales[ind2][-i-1])
                    new_plan_item_index1.append(dic_initial_index[ind1][i])
                    new_plan_item_index2.append(dic_initial_index[ind2][-i-1])
                    i += 1
            else: break   
            
    new_plan_ind1 = new_plan_ind1 + dic_initial_sales[ind1][i:]
    new_plan_item_index1 = new_plan_item_index1 + dic_initial_index[ind1][i:]
    if i == 0: 
        new_plan_ind2 = dic_initial_sales[ind2]
        new_plan_item_index2 = dic_initial_index[ind2]
    else: 
        new_plan_ind2 = new_plan_ind2 + dic_initial_sales[ind2][:-i]
        new_plan_item_index2 = new_plan_item_index2 + dic_initial_index[ind2][:-i]
    
    new_plan_group1[ind1] = sorted(new_plan_ind1, reverse=True)
    new_plan_group2[ind2] = sorted(new_plan_ind2, reverse=True)
    new_plan_group1_index[ind1] = sorted(new_plan_item_index1, reverse=True)
    new_plan_group2_index[ind2] = sorted(new_plan_item_index2, reverse=True)
    
    new_plan_group1_sum[ind1] = sum(new_plan_ind1)
    new_plan_group2_sum[ind2] = sum(new_plan_ind2)
    
    cur_imbalance_list.append(cur_imbalance)
    print(i, cur_imbalance)


sales_newer_mapping = sales_new_mapping.copy()
for ind in index1_used:
    sales_newer_mapping['initial_sum'].loc[ind] = new_plan_group1_sum[ind]
    sales_newer_mapping['initial_indi_sum'].loc[ind] = new_plan_group1[ind]
    sales_newer_mapping['initial_indi_index'].iloc[ind] = new_plan_group1_index[ind]

for ind in index2_used:
    sales_newer_mapping['initial_sum'].loc[ind] = new_plan_group2_sum[ind]
    sales_newer_mapping['initial_indi_sum'].loc[ind] = new_plan_group2[ind]
    sales_newer_mapping['initial_indi_index'].iloc[ind] = new_plan_group2_index[ind]
    
sales_newer_mapping_sorted = sales_newer_mapping.sort_values(by='initial_sum')

index_remain = list(set(list(sales_newer_mapping.index)) - set(index_used))

## there is error in total imbalance calculation here, needs correction
total_imbalance = sum([np.abs(j) for j in cur_imbalance_list]) + \
sum([np.abs(sales_newer_mapping.iloc[ind].initial_sum - mean_sales) for ind in index_remain])

#print('total imbalance:', total_imbalance)
if total_imbalance > total_imbalance_last_round:
    print ('needs intervention!')
    
else: 
    sales_new_mapping = sales_newer_mapping.copy()
    total_imbalance_last_round = total_imbalance
    

group1: 0 19
group2: -1 56
9426
5 -1683
group1: 1 20
group2: -2 40
8804
6 -1313
group1: 2 21
group2: -3 57
8740
6 -915
group1: 3 22
group2: -4 4
7316
4 7316
group1: 4 31
group2: -5 13
7231
6 7231
group1: 5 23
group2: -6 5
6497
5 6497
group1: 6 12
group2: -7 3
6309
1 -1297
group1: 7 29
group2: -8 44
5668
1 -220
group1: 8 2
group2: -9 9
5370
1 -857
group1: 9 25
group2: -10 15
5331
7 5331
group1: 10 24
group2: -11 59
5041
5 -269
group1: 11 35
group2: -12 1
4934
2 4934
group1: 12 37
group2: -13 8
4778
5 1161
group1: 13 52
group2: -14 14
4645
7 4645
group1: 14 48
group2: -15 11
4479
6 4479
group1: 15 32
group2: -16 27
4273
4 -59
group1: 16 53
group2: -17 17
4110
8 4110
group1: 17 33
group2: -18 41
3489
3 -322
group1: 18 54
group2: -19 7
3464
5 3464
group1: 19 38
group2: -20 16
3137
8 3137
group1: 20 51
group2: -21 10
2517
6 2517
group1: 21 39
group2: -22 42
2308
3 -311
group1: 22 47
group2: -23 6
1732
5 1732
group1: 23 55
group2: -24 46
1715
3 -19
group1: 24 30
group2: -25 26
1518
3 -728
gr

/Users/chao_zhuo/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
### 4th adjustment

sales_new_mapping_sorted = sales_new_mapping.sort_values(by='initial_sum')

# pick the station with the largest total sales so far specifically, for adjustment
working_over = sales_new_mapping_sorted.initial_indi_sum.iloc[-1]  
new_sa = []
new_saj_list = []
new_sa_index = []
new_saj_index_list = []
i = 0
j = 0
num_try = 59 # consider other 59 stations for switching items
cur_over = sum(working_over) - mean_sales
sa = working_over[i] 
sa_all_index = sales_new_mapping_sorted.initial_indi_index.iloc[-1]

while i < len(working_over):
    
    while (cur_over > 0) & (j < num_try):
        new_saj = []
        new_saj_index = []
        
        saj = sales_new_mapping_sorted.initial_indi_sum.iloc[j]  # consider jth station for adjustment 
        saj_index = sales_new_mapping_sorted.initial_indi_index.iloc[j]
        
        if (sa < min(saj)) & (i < len(working_over)):
            j += 1
            new_saj_list.append(saj)
            new_saj_index_list.append(saj_index)
        else:        
            cur_below = sum(saj) - mean_sales
            
            while (cur_below < 0) & (i < len(working_over)):
                
                # loop over items in jth station
                k = 0
                while (k < len(saj)) & (cur_below < 0):
                    sak = saj[k]
                    
                    if (sa > sak) & (i < len(working_over)):
                        new_sa.append(sak)
                        new_saj.append(sa)
                        new_sa_index.append(saj_index[k])
                        new_saj_index.append(sa_all_index[i])
                        
                        cur_over -= sa - sak
                        cur_below += sa - sak
                        i += 1
                        k += 1
                        if (i < len(working_over)):
                            sa = working_over[i]  # update item to consider next in station with largest sales
                        
                    else:
                        new_saj.append(sak)
                        new_saj_index.append(saj_index[k])
                        k += 1
                        
                
                else:
                    new_saj_list.append(new_saj + saj[k:])
                    new_saj_index_list.append(new_saj_index + saj_index[k:])
                    j += 1
                    break
            
            else: 
                new_saj_list.append(saj)
                new_saj_index_list.append(saj_index)
                j += 1
                break
                
    else: 
        new_sa += working_over[i:]
        new_sa_index += sa_all_index[i:]
        break                   


# create table to store updated items for all stations        
new_saj_station_index = [i for i in sales_new_mapping_sorted.index]
df_new_saj = pd.DataFrame({'station_index': new_saj_station_index[:59],
                           'initial_sum': [sum(j) for j in new_saj_list],
                           'initial_indi_sum': new_saj_list,
                           'initial_indi_index': new_saj_index_list}) 
df_new_saj.set_index('station_index', inplace=True)
add_last = [i for i in sales_new_mapping_sorted.index][-1]
df_new_saj.loc[add_last] = [sum(new_sa), new_sa, new_sa_index]
df_new_saj_sorted = df_new_saj.sort_values(by = 'initial_sum')
sales_new_mapping_sorted = df_new_saj_sorted.copy()

In [8]:
# station location mapping
workstation_location_mapping = df_ila[['workstation','location']]

# create new assignment table, same format as in excel
item_list = []
sale_list = []
station_list = []
location_list = []
chamber_list = []
for row in sales_new_mapping_sorted.iterrows():
    item_indexes = row[1][2]
    item_sale = row[1][1]
    station_index = row[0]
    for i,ind in enumerate(item_indexes):
        item = dic_index_item_mapping[ind]
        sale = item_sale[i]
        workstation = station_count_location_mapping['workstation'].iloc[station_index]
        location = workstation_location_mapping[workstation_location_mapping.workstation == \
                                                     workstation].iloc[i].location
        
        item_list.append(item)
        sale_list.append(sale)
        station_list.append(workstation)
        location_list.append(location)
        chamber_list.append('Chamber_1')
        
sample_df1 = pd.DataFrame({'item': item_list,
                           'sales': sale_list,
                           'location': location_list,
                           'workstation': station_list,
                           'chamber': chamber_list})

# write to txt
sample_df1.to_csv(r'balanced_item_location_assignment.txt', header=sample_df1.columns, index=None, sep='\t', mode='a')


In [9]:
# check new total imbalance
df1 = pd.read_csv('balanced_item_location_assignment.txt', sep = '\t')
df_sumsales = df1.groupby(['workstation'], as_index=False).agg({
    'sales': 'sum',
    'location': 'count'
})

df_sumsales['diffe'] = df_sumsales['sales'] - int(df_sumsales.sales.mean())

sum(np.abs(df_sumsales.diffe)), sum(df_sumsales.sales)

(148558, 1557926)